In [1]:
from flask import *
import sqlite3 as sql




app=Flask(__name__)
app.secret_key='login'




@app.route("/success", methods=["GET","POST"])
def success():
    
    print(session.get("username"))
    if(session.get("username") == None):
        print("No User Logged In")
        return redirect(url_for("login"))
    
    
    username= session["username"]
    return render_template("result.html",message=msg,loginuser=username)


@app.route("/fail/<msg>", methods=["GET","POST"])
def fail(msg):
    return render_template("result.html",message=msg)



@app.route("/homepage",methods=["GET","post"])
def homepage():
    
    print(session.get("username"))
    
      
    cnt = session.get("count")
    
    if cnt == None:
        cnt = 1
    else:
        cnt = int(cnt) + 1
    
    print(cnt)
    
    
    if(session.get("username") == None):
        print("No User Logged In")
        return redirect(url_for("login",message=msg))
    
    session["count"]=cnt
    username= session["username"]
    resp = (render_template("homepage.html",loginuser=username,count=cnt))
   
    return resp

@app.route("/profilepage",methods=["GET","post"])
def profilepage():
    '''username= session["username"]
    return render_template("profilepage.html",loginuser=username)'''
    if 'loggedin' in session:
        print ("a - "+session["username"])
        username= session["username"]
        with sql.connect("database.db") as con:
                cur = con.cursor() 
                cur.execute('SELECT * FROM register WHERE name = ?', [session['username']])
                user = cur.fetchone()
                print(user)
                return render_template('profilepage.html', userinfo=user)
    return redirect(url_for('login'))


@app.route("/profileupdatepage",methods=["GET","post"])
def profileupdatepage():
        return render_template("profileupdatepage.html")


@app.route("/changepassword",methods=["GET","post"])
def changepassword():
    return render_template("changepassword.html")
 
@app.route("/registerpage",methods=["GET","post"])
def register():
    return render_template("registerpage.html")
 


@app.route("/process", methods=["GET","POST"])
def process():
    '''tuser=request.form["tuser"]
    tpass=request.form["tpass"]
   '''
    '''if(tuser=="kalpak" and tpass=="kalpak"):
        session["username"]=tuser
        return redirect(url_for("homepage",msg="login successfull"))
        
    else:
        return redirect(url_for("fail",msg="sorry,login invalid !!"))
    '''
    
    if request.method == 'POST':
        tuser=request.form["tuser"]
        tpass=request.form["tpass"]
        with sql.connect("database.db") as con:
                cur = con.cursor()
                cur.execute('SELECT * FROM register WHERE name = ? AND pass = ?', (tuser, tpass))
                user = cur.fetchone()
                if user:
                    session['loggedin'] = True
                    session['username'] = tuser
                    return redirect(url_for("homepage",msg="login successfull"))
                else:
                    return redirect(url_for("fail",msg="sorry,login invalid !!"))
    
    return render_template('result.html', message=msg)
    
    
    
    
    
    
@app.route('/addrec',methods = ['POST', 'GET'])
def addrec():
    if request.method == 'POST':
        
        try:
            
            nm = request.form['name']
            email = request.form['temail']
            mobile = request.form['tmobile']
            city = request.form['tcity']
            passw = request.form['pass']
            cpass = request.form['cpass']
            with sql.connect("database.db") as con:
                cur = con.cursor()
                cur.execute("INSERT INTO register (name,email,mobile,city,pass,cpass) VALUES (?,?,?,?,?,?)",(nm,email,mobile,city,passw,cpass) )
                con.commit()
                msg = "Record successfully added"
        except:
            con.rollback()
            msg = "error in insert operation"
      
        finally:
            return render_template("result.html",message = msg)
            con.close()
            
@app.route('/updaterec',methods = ['POST', 'GET'])
def updaterec(): 
    if 'loggedin' in session:
        username=[session['username']]
        if request.method == 'POST':
            try:
                nm = request.form['name']
                email = request.form['temail']
                mobile = request.form['tmobile']
                city = request.form['tcity']
                with sql.connect("database.db") as con:
                    cur = con.cursor()
                    cur.execute("UPDATE register SET name=?, email=?,mobile=?,city=? WHERE name=?",(nm,email,mobile,city,username) )
                    con.commit()
                    msg = "Record successfully Updated"
            except:
                con.rollback()
                msg = "error in update operation"
      
            finally:
                  return render_template("result.html",message=msg)
                  con.close()
            
@app.route('/list')
def list():
    
    con = sql.connect("database.db")
    con.row_factory = sql.Row
   
    cur = con.cursor()
    cur.execute("select * from register")
   
    rows = cur.fetchall();
    return render_template("list.html",rows = rows)
            
            


@app.route("/logout")
def logout():
    session.pop("username")
    return redirect(url_for("login"))


@app.route("/")
def login():
    
       
        print(session.get("username"))
        if(session.get("username") != None ):
            print("User Already Logged In!")
            return redirect(url_for("homepage"))
    
        return render_template("login.html")
    

class ServerError(Exception):pass

    
    
if __name__=="__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
